# Spectrogram and spectrum slice display

Select an audio file from the dropdown list to display its waveform spectrogram. Click on a part of the waveform or spectrogram to select a spectrum slice to display. Mouse over the spectrum slice to explore the frequencies. The circle tracks the current frequency in the spectrum slice and spectrogram, and current values are displayed in the table.

Spectrogram and spectrum slice values below the low threshold are shaded blue. The analysis window slider adjusts the spectrogram analysis window.

In [ ]:
from spectrum_slice_app import spectrum_slice_app
from bokeh_phon.utils import remote_jupyter_proxy_url_callback, set_default_jupyter_url
from bokeh.io import show
# Set this value to the hostname you find in your browser url bar. Do not include
# any path found after the hostname.
set_default_jupyter_url('https://hub.gke.mybinder.org/')
# If running locally, exclude the notebook_url parameter.
# show(spectrum_slice_app)     # Run in local jupyter notebook (not in jupyterhub/binderhub)
show(spectrum_slice_app, notebook_url=remote_jupyter_proxy_url_callback)